In [2]:
# Eval code, adapted for jupyter notebook with backtest decorator
import numpy as np
import pandas as pd
from functools import wraps

nInst = 0
nt = 0
commRate = 0.0005
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep=r'\s+', header=None, index_col=None)
    (nt,nInst) = df.shape
    return (df.values).T

pricesFile="./.data/2025/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(getPosition, prcHist=None, numTestDays=200):
    if prcHist is None:
        prcHist = prcAll
    
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt_local) = prcHist.shape
    startDay = nt_local + 1 - numTestDays
    
    for t in range(startDay, nt_local+1):
        prcHistSoFar = prcHist[:,:t]
        curPrices = prcHistSoFar[:,-1]
        if (t < nt_local):
            # Trading, do not do it on the very last day of the test
            newPosOrig = getPosition(prcHistSoFar)
            posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
            newPos = np.clip(newPosOrig, -posLimits, posLimits)
            deltaPos = newPos - curPos
            dvolumes = curPrices * np.abs(deltaPos)
            dvolume = np.sum(dvolumes)
            totDVolume += dvolume
            comm = dvolume * commRate
            cash -= curPrices.dot(deltaPos) + comm
        else:
            newPos = np.array(curPos)
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        if (t > startDay):
            print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
            todayPLL.append(todayPL)
    
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(249) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

def backtest(numTestDays=200, prcHist=None, verbose=True):
    """
    Decorator to backtest a position function
    
    Args:
        numTestDays: Number of days to test (default 200)
        prcHist: Price history array (default uses global prcAll)
        verbose: Whether to print detailed results (default True)
    """
    def decorator(getPosition_func):
        @wraps(getPosition_func)
        def wrapper(*args, **kwargs):
            print(f"Testing function: {getPosition_func.__name__}")
            
            # Run the backtest
            (meanpl, ret, plstd, sharpe, dvol) = calcPL(
                getPosition_func, 
                prcHist if prcHist is not None else prcAll, 
                numTestDays
            )
            
            score = meanpl - 0.1*plstd
            
            if verbose:
                print ("=====")
                print (f"Function: {getPosition_func.__name__}")
                print ("mean(PL): %.1lf" % meanpl)
                print ("return: %.5lf" % ret)
                print ("StdDev(PL): %.2lf" % plstd)
                print ("annSharpe(PL): %.2lf " % sharpe)
                print ("totDvolume: %.0lf " % dvol)
                print ("Score: %.2lf" % score)
                print ("=====")
            
            # Store results as function attributes for later access
            wrapper.results = {
                'meanpl': meanpl,
                'return': ret,
                'plstd': plstd,
                'sharpe': sharpe,
                'dvol': dvol,
                'score': score
            }
            
            return wrapper.results
        
        # Run backtest immediately when decorator is applied
        wrapper()
        return wrapper
    
    return decorator



Loaded 50 instruments for 750 days


In [3]:
# Bit of a dumb/crazy idea but can we just ask an llm to think and make decisions for us?
import ollama
import numpy as np
import json

model = "llama3.2:3b"  # Using a more stable model

def get_llm_trading_decision(price_history, stock_id, current_day):
    """
    Ask LLM to analyze price data and make trading decisions
    """
    # Prepare price data for the LLM
    recent_prices = price_history[-10:].tolist()  # Last 10 days
    price_change = (price_history[-1] - price_history[-2]) / price_history[-2] * 100
    
    # Calculate some basic stats
    avg_price = np.mean(recent_prices)
    volatility = np.std(recent_prices)
    trend = "up" if recent_prices[-1] > recent_prices[0] else "down"
    
    prompt = f"""
You are a quantitative trader analyzing stock {stock_id} on day {current_day}.

Recent price data (last 10 days): {recent_prices}
Today's price change: {price_change:.2f}%
10-day average: {avg_price:.2f}
Volatility (std dev): {volatility:.2f}
Overall trend: {trend}

Based on this data, decide your position for this stock:
- Return a single integer between -10 and +10
- Positive = long position, Negative = short position, 0 = no position
- Consider momentum, mean reversion, volatility, and trend
- Be decisive but not reckless

Respond with ONLY a single integer, no explanation.
"""

    try:
        response = ollama.generate(model=model, prompt=prompt)
        decision_text = response['response'].strip()
        
        # Extract number from response
        import re
        numbers = re.findall(r'-?\d+', decision_text)
        if numbers:
            decision = int(numbers[0])
            return np.clip(decision, -10, 10)  # Ensure within bounds
        else:
            return 0  # Default to no position if can't parse
            
    except Exception as e:
        print(f"Error getting LLM decision for stock {stock_id}: {e}")
        return 0

@backtest()
def llm_trading_strategy(prcHistSoFar):
    """
    Trading strategy that uses LLM to make decisions for each stock
    """
    nInst, nt = prcHistSoFar.shape
    
    if nt < 5:  # Need some history
        return np.zeros(nInst)
    
    positions = np.zeros(nInst)
    
    print(f"Consulting LLM for trading decisions on day {nt}...")
    
    # Get LLM decision for each stock
    for stock_id in range(nInst):
        stock_prices = prcHistSoFar[stock_id, :]
        decision = get_llm_trading_decision(stock_prices, stock_id, nt)
        positions[stock_id] = decision
        
        # Optional: print some decisions for debugging
        if stock_id < 5:  # Only print first few
            print(f"Stock {stock_id}: Position {decision}")
    
    return positions

Testing function: llm_trading_strategy
Consulting LLM for trading decisions on day 551...
Stock 0: Position 8
Stock 1: Position 6
Stock 2: Position 5
Stock 3: Position 3
Stock 4: Position 5
Consulting LLM for trading decisions on day 552...
Stock 0: Position 4
Stock 1: Position 3
Stock 2: Position 5
Stock 3: Position 5
Stock 4: Position 5
Day 552 value: -12.79 todayPL: $-6.89 $-traded: 16096 return: -0.00079
Consulting LLM for trading decisions on day 553...
Stock 0: Position 5
Stock 1: Position 5
Stock 2: Position 3
Stock 3: Position 3
Stock 4: Position 6
Day 553 value: -11.77 todayPL: $1.02 $-traded: 20439 return: -0.00058
Consulting LLM for trading decisions on day 554...
Stock 0: Position 3
Stock 1: Position 5
Stock 2: Position 5
Stock 3: Position 5
Stock 4: Position 5
Day 554 value: -28.24 todayPL: $-16.47 $-traded: 25506 return: -0.00111
Consulting LLM for trading decisions on day 555...
Stock 0: Position 5
Stock 1: Position 3
Stock 2: Position 5
Stock 3: Position 5
Stock 4: Posi

KeyboardInterrupt: 